In [1]:
import pygame as pg
import sys
import os 
import time 
import utils as ut
import animations as an
import pyrow.pyrow as pr
from threading import Thread

# Standard 
BLUE = (0,80,239)
BLACK = (0,0,0)
GREEN = (0,138,0)
RED = (229,20,0)
WHITE = (255,255,255)
GOLD = (248,217,73)
SILVER = (170,169,173)

# Initializing
pg.init()

# Importing all images/fonts
[flag_be, flag_fr, flag_rma, flag_x] = ut.prepare_flags()
boat = pg.transform.scale(pg.image.load(os.path.join(".", "figures","warship.png")),(375,375))
background_image = pg.transform.scale(pg.image.load(os.path.join(".", "figures","canal.png")),(4000,2000))
[font_10, font_30, font_50, font_70, font_90] = [ut.import_fonts(size) for size in [10,30,50,70,90]]

# Screen parameters
clock = pg.time.Clock()
fps = 30
screen = pg.display.set_mode((1600,900),pg.SCALED)
width = screen.get_width()
height = screen.get_height()
pg.display.set_caption("Chapelié")
pg.display.set_icon(boat)

# Assigning all utilites
speed = 4
colorswapper_rma = ut.ColorSwapper([WHITE,GREEN],.2,10)
colorswapper_x = ut.ColorSwapper([WHITE,GREEN],.2,10)
background = an.Background(screen,background_image,speed)
boat_rma = an.Boat(screen,boat,flag_be,175)
boat_x = an.Boat(screen,boat,flag_fr,-75)
flag_rma = an.AnimatedFlag(screen,flag_rma,(-100-flag_rma[0].get_width(),flag_rma[0].get_height()//2))
flag_x = an.AnimatedFlag(screen,flag_x,(100,flag_x[0].get_height()//2))

# Distance to screen x location
location = lambda x: 1 - 1/(x/1000 + 1)

pygame 2.5.2 (SDL 2.28.2, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Game part

In [3]:
def options(options_time:int):
    continue_color = WHITE
    back_color = WHITE
    s = pg.Surface((width,height))
    s.set_alpha(200)
    s.fill((0,0,0))
    screen.blit(s,(0,0))
    while True:
        continue_button = an.button(screen,"Continue",font_50,(width//2,height//2-50),(375,75),continue_color)
        back_button = an.button(screen,"Quit",font_50,(width//2,height//2+50),(375,75),back_color)
        mx,my = pg.mouse.get_pos()
        for event in pg.event.get():
            if continue_button.collidepoint(mx,my):
                continue_color = RED
                if event.type == pg.MOUSEBUTTONDOWN:
                    return time.time() - options_time
            else:
                continue_color = WHITE
            if back_button.collidepoint(mx,my):
                back_color = RED
                if event.type == pg.MOUSEBUTTONDOWN:
                    return "quit"
            else:
                back_color = WHITE
        pg.display.flip()
        clock.tick(fps)

In [4]:
def game(total_time:int,machines:list[pr.PyErg]):
    for machine in machines:
        machine.set_clock()
    machines_rma = machines[:len(machines)//2]
    machines_x = machines[len(machines)//2:]
    countdown = an.Countdown(screen,time.time(),total_time,font_50,WHITE)
    options_color = WHITE
    count = 0
    distance_rma = 0
    distance_x = 0
    delta = 0
    start = time.time()
    duration = 0
    while duration < total_time:
        # Time
        duration = time.time()-start

        # Retrieve distances. RMA as reference
        try:
            distance_rma = sum([machine.get_monitor()["distance"] for machine in machines_rma])
            distance_x = sum([machine.get_monitor()["distance"] for machine in machines_x])
            delta = distance_rma-distance_x
        except:
            print("DISTANCE ERROR")

        # Swapping colors
        color_rma = colorswapper_rma.swap(distance_rma,400,duration)
        color_x = colorswapper_x.swap(distance_x,400,duration)

        # Rendering of elements
        background.blit(min(distance_rma,distance_x))
        flag_rma.blit()
        flag_x.blit()
        boat_rma.blit(location(max(0,delta))*width)
        boat_x.blit(location(max(0,-delta))*width)
        countdown.blit()

        an.render_text(screen,font_30,"VS",(width//2,height//2))
        an.render_text(screen,font_50,str(round(distance_rma)) + "m",(width//2+400,height//2),color=color_rma)
        an.render_text(screen,font_50,str(round(distance_x)) + "m",(width//2-400,height//2),color=color_x)
        an.render_text(screen,font_10,"Bouzin Philippot Prakopetz Vancanneyt (174 POL)",(200,10))

        # Options an.button
        options_button = an.button(screen,"Options",font_30,(width-150,50),(250,50),options_color)
        mx,my = pg.mouse.get_pos()
        for event in pg.event.get():
            if options_button.collidepoint(mx,my):
                options_color = RED
                if event.type == pg.MOUSEBUTTONDOWN:
                    return_options = options(time.time())
                    if return_options == "quit":
                        return return_options
                    else:
                        countdown.start += return_options
            else:
                options_color = WHITE
        
        # Pygame stuff
        pg.display.flip()
        clock.tick(fps)
        screen.fill(BLACK)
        count += 1
    return [[distance_rma,boat_rma],[distance_x,boat_x]]

## Playing

In [9]:
def main():
    while True:
        return_menu = main_menu()
        if return_menu == "play":
            return_play = play_menu()
            if type(return_play) == int:
                return_connect = connect_menu()
                if type(return_connect) == list:
                    return_game = game(return_play,return_connect)
                    if return_game == "quit":
                        continue
                    else:
                        end_menu(return_game)
                else:
                    continue
            else:
                continue
        else:
            quit()
#! I am becoming schizophrenic from this code, but no worries

## Running

In [2]:
import pygame as pg
import sys
import os 
import utils as ut
import animations as an

# Standard 
BLUE = (0,80,239)
BLACK = (0,0,0)
GREEN = (0,138,0)
RED = (229,20,0)
WHITE = (255,255,255)
GOLD = (248,217,73)
SILVER = (170,169,173)
def main():
    # Initializing
    pg.init()

    # Importing all images/fonts
    [flag_be, flag_fr, flag_rma, flag_x] = ut.prepare_flags()
    boat = pg.transform.scale(pg.image.load(os.path.join(".", "figures","warship.png")),(375,375))
    boat_be = pg.transform.scale(pg.image.load(os.path.join(".", "figures","warshipbe.png")),(375,375))
    boat_fr = pg.transform.scale(pg.image.load(os.path.join(".", "figures","warshipfr.png")),(375,375))
    background_image = pg.transform.scale(pg.image.load(os.path.join(".", "figures","canal.png")),(4000,2000))
    fonts = [ut.import_fonts(size) for size in [10,30,50,70,90]]

    # Screen parameters
    clock = pg.time.Clock()
    fps = 30
    screen = pg.display.set_mode((1600,900),pg.SCALED)
    pg.display.set_caption("Chapelié")
    pg.display.set_icon(boat)

    # Assigning all utilites
    speed = 4
    colorswapper_rma = ut.ColorSwapper([WHITE,GREEN],.2,10)
    colorswapper_x = ut.ColorSwapper([WHITE,GREEN],.2,10)
    background = an.Background(screen,background_image,speed)
    boat_rma = an.Boat(screen,boat_be,flag_be,175)
    boat_x = an.Boat(screen,boat_fr,flag_fr,-75)
    flag_rma = an.AnimatedFlag(screen,flag_rma,(-100-flag_rma[0].get_width(),flag_rma[0].get_height()//2))
    flag_x = an.AnimatedFlag(screen,flag_x,(100,flag_x[0].get_height()//2))

    game = an.Menu(screen,fonts,clock,fps,background,boat_x,boat_rma,[flag_x,flag_rma],[colorswapper_x,colorswapper_rma])

    while True:
        return_menu = game.main()
        if return_menu == "play":
            return_play = game.play()
            if type(return_play) == int:
                return_connect = game.connect()
                if type(return_connect) == list:
                    return_game = game.game(return_play,return_connect)
                    if return_game == "quit":
                        continue
                    else:
                        game.end(return_game)
                else:
                    continue
            else:
                continue
        else:
            pg.display.quit()
            pg.quit()
            sys.exit()
            
main()

pygame 2.5.2 (SDL 2.28.2, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/p3tzl/Documents/School/RMA_MA2/Chapelier/src/pyrow/pyrow.py:169: UserWarning: DEBUG: usb kernel driver not on linux
  warn("DEBUG: usb kernel driver not on {}".format(sys.platform))
/home/p3tzl/Documents/School/RMA_MA2/Chapelier/src/pyrow/pyrow.py:181: UserWarning: DEBUG: usb error whilst setting configuration, [Errno 16] Resource busy
  warn("DEBUG: usb error whilst setting configuration, {}".format(e))


USBError: [Errno 16] Resource busy

: 

## Test for projector 

In [2]:
import pyrow.pyrow as pr 
available_machines = [pr.PyErg(machine) for machine in list(pr.find())]

print(available_machines[0].get_monitor()["distance"])

/home/p3tzl/Documents/School/RMA_MA2/Chapelier/src/pyrow/pyrow.py:180: UserWarning: DEBUG: usb error whilst setting configuration, [Errno 16] Resource busy
  warn("DEBUG: usb error whilst setting configuration, {}".format(e))


ConnectionError: USB device disconected